Данный обзор основан на материале обзорной статьи ([Zhang, 2021]($Dive into Decision Trees and Forests: A Theoretical Demonstration$)), который будет дополнен материалами из других источников и программным кодом.

Первый вариант плана:
- простое дерево для бинарной классификации, правило построения
- дерево для регрессии, мультиклассовой классификации, для анализа выбросов
- органичения на кол-во листьев и глубину
- прунинг
- стохастичность (на уровне вершин и деревьев)
----
Второй вариант плана:
- принципы построения деревьев: критерий разделения, решающие правила, количество листьев
- работа с категориальными признаками
- алгоритмы ID3, CART и C4.5
- ансамблирование
- XGBoost, LightGBM, CatBoost
----
Третий вариант плана:
- простое дерево для бинарной классификации
- дерево в общем виде: критерий разделения, решающие правила, количество листьев + пара слов об ансамблировании
- разновидности деревьев, свойства (например, связь с KNN, влияние нормализации и пр.)
- бустинг деревьев
- XGBoost, LightGBM, CatBoost
----
Чем отличаются деревья:
- способ разделения (gini, entropy, general loss function)
- решающая функция и тип задачи (loss)
- ограничение на размер (макс. глубина, макс. листьев), способ прунинга
- способ работы с пропущенными данными
----

Из Pattern Recognition and Machine Learning:

For instance, to predict a patient’s disease, we might first ask “is their temperature greater
than some threshold?”. If the answer is yes, then we might next ask “is their blood
pressure less than some threshold?”. Each leaf of the tree is then associated with a
specific diagnosis.

it is found empirically that often
none of the available splits produces a significant reduction in error, and yet after
several more splits a substantial error reduction is found. For this reason, it is common practice to grow a large tree, using a stopping criterion based on the number
of data points associated with the leaf nodes, and then prune back the resulting tree.
The pruning is based on a criterion that balances residual error against a measure of
model complexity.

(представим, что в бинарной классификации с 2 признаками объекты класса 0 лежат в квадранте x>0, y>0 и в квадранте x<0, y<0)

----

Оптимальное решающее дерево - это? 100%-я точность не ознчает оптимальность. Оптимальность - хорошая генерализация. Предположив, что лучше генерализуются простые гипотезы, нужно уменьшить средний путь в дереве по всему датасету. Эта задача NP-полная (Constructing Optimal Binary Decision Trees is NP-Complete)

Разделение по координатным осям, конечно, не всегда оптимально. Также функция получается разнывной.

При label-кодировании категориальных признаков некоторые категории всегда будут в разных сплитах
Представим, что есть одна важная категория. При label-кодировании вместе с ней в ветвь попадет много "мусора".

https://www.quora.com/For-decision-trees-if-your-features-are-categorical-do-you-need-to-one-hot-encode-them-or-just-replace-them-with-integers

По поводу target-кодирования две проблемы: во-первых представим, что есть только по 1 экземпляру каждого класса; во-2 признаки могут влиять на таргет не независимо

TODO вставить информацию про градиентный спуск и переработать (метод Ньютона)

----

Из Classification and regression trees http://pages.stat.wisc.edu/~loh/treeprogs/guide/wires11.pdf

variables that have more distinct values have a greater chance to be selected. This
selection bias affects the integrity of inferences drawn
from the tree structure.
Building on an idea that originated in the FACT
algorithm, CRUISE, GUIDE, and QUEST use
a two-step approach based on significance tests to
split each node. First, each X is tested for association
with Y and the most significant variable is selected.
Then, an exhaustive search is performed for the set
S. Because every X has the same chance to be selected if each is independent of Y, this approach is
effectively free of selection bias. Besides, much computation is saved as the search for S is carried out
only on the selected X variable. GUIDE and CRUISE
use chi squared tests, and QUEST uses chi squared
tests for unordered variables and analysis of variance
(ANOVA) tests for ordered variables. CTree, another unbiased method, uses permutation tests.

CHAID employs yet another strategy. If X is
an ordered variable, its data values in the node are
split into 10 intervals and one child node is assigned
to each interval. If X is unordered, one child node
is assigned to each value of X. Then, CHAID uses
significance tests and Bonferroni corrections to try to
iteratively merge pairs of child nodes. This approach
has two consequences. First, some nodes may be split
into more than two child nodes. Second, owing to the
sequential nature of the tests and the inexactness of
the corrections, the method is biased toward selecting
variables with few distinct values.

Because the total model complexity is shared between the tree structure and the set of node models,
the complexity of a tree structure often decreases as
the complexity of the node models increases. Therefore, the user can choose a model by trading off tree
structure complexity against node model complexity.

----

Из XGBoost: A Scalable Tree Boosting System https://arxiv.org/abs/1603.02754

Формула для loss-функции (после материала о бустинге)

Формула для оптимизации 2-го порядка

формула для расчета loss reduction

the algorithm first proposes candidate splitting points according to percentiles of feature
distribution (a specific criteria will be given in Sec. 3.3).
The algorithm then maps the continuous features into buckets split by these candidate points, aggregates the statistics
and finds the best solution among proposals based on the
aggregated statistics.

The global variant proposes all
the candidate splits during the initial phase of tree construction, and uses the same proposals for split finding at all levels. The local variant re-proposes after each split. The global
method requires less proposal steps than the local method.
However, usually more candidate points are needed for the
global proposal because candidates are not refined after each
split. The local proposal refines the candidates after splits,
and can potentially be more appropriate for deeper trees.

we propose to add a default
direction in each tree node, which is shown in Fig. 4. When
a value is missing in the sparse matrix x, the instance is
classified into the default direction.

### Структура решающих деревьев

Решающие деревья применяются в основном в задачах классификации и регрессии в [машинном обучении]($Введение в машинное обучение$) на табличных данных. В общем виде решающее дерево - это иерархическая схема принятия решений в виде графа (*рис. 1а*). В промежуточных вершинах (звеньях) проверяются некие условия, и в зависимости от результатов выбирается путь в графе, который приводит к одной из конечных вершин (листьев).

**(рис. 1)**

В общем случае звенья и листья могут быть сложными функциями. Однако в большинстве практических реализаций (scikit-learn, XGBoost, CatBoost **???**) каждый лист соответстствует константному ответу, а в каждом звене проверяется значение лишь одного признака (*рис. 1b*). Если это количественный признак, то все значения больше некоего порога отправляются по одному пути, меньше порога - по другому пути. Если это категориальный признак, то одна или несколько категорий отправляется по одному пути, остальные категории по другому пути.

*Примечание. В CatBoost категориальные признаки иногда преобразуются в количественные, более подробно рассмотрим позже.*

С первого взгляда кажется, что такой подход хорошо подходит для одних задач, но очень плохо подходит для других задач.

**Позитивный пример.** Многие способы принятия решений выглядят как блок-схема. Например при постановке диагноза врач сначала может измерить температуру. Если температура выше определенного порога, то может заподозрить простуду и посмотреть горло, послушать легкие и так далее. Листьями решающего дерева окажутся конкретные диагнозы. В целом, решающее дерево хорошо следует идее о том, что на некоторые признаки нужно обращать внимание только при условии определенных значений других признаков.

**Негативный пример.** Попробуем аппроксимировать функцию $y(x) = x_2 - x_1$ (*рис. 2a*) решающим деревом (*рис. 2b*). Решающее дерево разбивает все пространство признаков на прямоугольные области, в каждой из которых ответом является константа. В данной задаче такой подход явно неоптимален. Во-первых, если решающее дерево представить в виде функции $f(x)$, то эта функция будет разрывной (кусочно-постоянной), тогда как целевая функция $y(x)$ непрерывна. Во-вторых, на ответ влияет разность $x_2 - x_1$, а в каждом звене дерева проверяется либо $x_1$, либо $x_2$, что тоже неоптимально. Для достижения хорошей точности потребуется большое и сложное дерево, хотя сама задача определения $y$ очень проста. И наконец, поскольку решающее дерево обучается на данных (о способе обучения мы поговорим далее), то в тех областях, где мало данных или они вовсе отсутствуют, функция $f(x)$ будет продолжена неправильно (проблема экстраполяции).

<img src="assets/trees1.png" width="1000" align="center">

<center><i>Рис. 2. Приближение непрерывной функции одним решающим деревом и ансамблем решающих деревьев.</i></center>

Проблему разрывности функции $f(x)$ и негладкости границ между листьями можно смягчить, если обучать не одно решающее дерево, а много деревьев, и усреднять их ответы (*рис. 2c*). При этом деревья либо должны обучаться на разных подвыборках данных, либо сам процесс обучения должен содержать элемент случайности, чтобы деревья получились неодинаковыми. Поэтому на практике обучают так называемый "случайный лес" из множества деревьев с помощью алгоритмов бэггинга или бустинга, которые мы расмотрим позже. Проблема экстраполяции при этом все же остается - например, случайный лес не может предсказывать значения, большие или меньшие всех тех, что встречались при обучении.

На нашей модельной задаче (*рис. 2*) решающие деревья дают не впечатляющий результат, тогда как, например, нейронная сеть в этой же задаче обучается почти точно аппроксимировать целевую функцию. Но на реальных задачах, наоборот, ансамбли решающих деревьев сейчас являются одними из самых эффективных моделей.

### Обучение решающих деревьев

Допустим мы имеем обучающую выборку из пар $\{x_i, y_i\}_{i=1}^N$ и хотим построить по нему решающее дерево. Можно без труда построить сколько угодно деревьев, дающих на обучающей выборке 100%-ю точность (если в ней нет примеров с одинаковыми $x$, но разными $y$). Для этого достаточно в каждой вершине выбирать для разделения любой признак и любой порог, и тогда рано или поздно все примеры попадут в разные листья. Но нам важна не точность на обучающей выборке, а степень обобщения - то есть точность на всем порождающем распределении, из которого взята обучающая выборка.

Можно воспользоваться принципом бритвы Оккама, который говорит, что простые гипотезы предпочтительнее сложных, и попробовать построить как можно более простое дерево. Однако задача нахождения наиболее простого решающего дерева для данного датасета (по суммрному количеству листьев или по средней длине пути в графе) является NP-полной ([Hancock et al., 1996]($Lower Bounds on Learning Decision Lists and Trees$), [Hyafil and Rivest, 1976]($Constructing optimal binary decision trees is NP-complete$)), то есть (если $P \neq NP$) экспоненциально сложной.

Вообще многие задачи являются экспоненциально сложными, если искать лучшее решение из всех возможных, то есть выполнять исчерпывающий поиск (exhaustive search). Но это не мешает находить хорошие приближенные решения, выполняя либо жадный поиск (greedy search), либо лучевой поиск (beam search). Жадный поиск означает, что на каждом шаге мы ищем локально оптимальное решение, то есть решение, приводящее к наибольшему "сиюминутному" выигрышу. Например, знакомясь с девушкой, которая вам нравится, вы не можете перебрать все возможные варианты развития диалога и заранее выбрать наилучший (т. е. выполнить исчерпывающий поиск), но можете после каждой фразы искать оптимальное продолжение диалога (жадный поиск).

*Примечание: пока работаем с количественными признаками, работу с категориальными признаками рассмотрим позднее.*

**Построение решающего дерева**

Применительно к решающим деревьям жадный поиск означает, что мы строим дерево пошагово, на каждом шаге заменяя один из листьев на разделяющее правило, ведущее к двум листьям. На каждом шаге мы должны выбрать:

1. Лист $L$, который заменяем решающим правилом
2. Способ разделения (признак и порог)
3. Значения $C$ и $C^\prime$ на двух новых листьях

Этот выбор мы должны сделать так, чтобы функция потерь всего дерева на обучающей выборке уменьшилась как можно сильнее. Для этого достаточно рассмотреть функцию потерь на примерах из обучающей выборки, которые относятся к листу $L$.

Шаг алгоритма выглядит следующим образом: мы перебираем все возможные листья, признаки и пороги (если обучающая выборка имеет размер $N$, то каждый признак принимает не более $N$ различных значений, поэтому для каждого признака имеет смысл перебирать не более $N-1$ порогов). Для каждого листа, признака и порога определяем значения $C$ и $C^\prime$ на листьях и считаем функцию потерь получившегося дерева. В итоге мы выбираем тот лист, признак и порог, для которых значение функции потерь наименьшее, и дополняем дерево новым разделяющим правилом.

Осталось ответить на вопрос: как именно определяются значения $C$ и $C^\prime$ на двух новых листьях?

Пусть после добавления разделяющего правила в один лист попало $N_1$ примеров со значениями целевого признака $y_1, \dots, y_{N_1}$, в другой лист попало $N_2$ примеров со значениями целевого признака $y^\prime_1, \dots, y^\prime_{N_2}$. Запишем суммарную функцию потерь после разделения, и минимизируем ее по $C$ и $C^\prime$:

$\sum\limits_{i=1}^{N_1} {Loss}(y_i, C) + \sum\limits_{i=1}^{N_2} {Loss}(y^\prime_i, C^\prime) \to \underset{C, C^\prime}{\min} \tag{1}$

Первое слагаемое зависит только от $C$, второе только от $C^\prime$, поэтому их можно минимизировать независимо.

**Оптимальное разделение в задаче регрессии**

В качестве функции потерь выберем среднеквадратичную ошибку. Исходя из формулы $(1)$, на первом листе нужно выбрать такое значение $C$ на первом листе, что:

$\sum\limits_{i=1}^{N_1} (C - y_i)^2 \to \underset{C}{\min}$

Поскольку $y_i$ - константы, то задача означает поиск минимума функции от одной переменной. Взяв производную и приравняв ее к нулю мы найдем, что $C$ - среднее арифметическое значений $y_i$. Аналогично, $C^\prime$ - среднее арифметическое значений $y_i^\prime$.

Если бы мы минимизировали не среднеквадратичную ошибку (MSE), а среднюю абсолютную ошибку (MAE), тогда оптимальным значением $C$ была бы медиана значений $y_i$.

**Оптимальное разделение в задаче классификации**

Мы можем пойти стандартным путем: пусть каждый лист выдает вероятности классов, а в качестве функции потерь используем категориальную перекрестную энтропию (logloss). Пусть количество классов равно $K$. Введем следующие обозначения:

- $\{p_k\}_{i=1}^{K}$ - доля объектов $k$-го класса в первом листе
- $\{p_k^\prime\}_{i=1}^{K}$ - доля объектов $k$-го класса во втором листе
- $\{C_k\}_{i=1}^{K}$ - предсказываемая вероятность $k$-го класса в первом листе
- $\{C_k^\prime\}_{i=1}^{K}$ - предсказываемая вероятность $k$-го класса во втором листе

Минимизируем суммарную ошибку на первом листе. Количество примеров в первом листе, для которых верным ответом является класс $k$, равно $N_1 p_k$. Суммируем ошибку по всем классам и будем искать минимум по $C_1, \dots, C_K$:

$ - \sum\limits_{k=1}^{K} N_1 p_k \log C_k \to \underset{C_1, \dots, C_K}{\min} \qquad \sum\limits_{k=1}^{K} C_k = 1$

Из данной формулы можно исключить $N_1$, и тогда задача нахождения оптимального распределения $C_k$ сводится к минимизации перекрестной энтропии между распределениями $p_k$ и $C_k$. Это эквивалентно минимизации расхождения Кульбака-Лейблера между $p_k$ и $C_k$ и достигается при $p_k = C_k$ для всех $k$. Полученный результат интуитивно понятен: если, например,  в листе 10% примеров класса 0 и 90% примеров класса 1, то предсказывая ответ для неизвестного примера оптимальным вариантом будет назначить классам вероятности 10% и 90%.

Таким образом, мы нашли, что оптимальное $C_k$ равно $p_k$. Суммарная функция потерь на обоих листьях будет равна:

$ - N_1 \sum\limits_{k=1}^{K} p_k \log p_k - N_2 \sum\limits_{k=1}^{K} p_k^\prime \log p_k^\prime = N_1 H(p_k) + N_2 H(p_k^\prime)$

Здесь $H(p_k)$ - энтропия распределения $p_k$ (ее более корректно записывать как $H(\{p_k\}_{i=1}^{K})$). Иными словами, нам нужно искать такой признак и порог, чтобы минимизировать взвешенную сумму энтропий распределений классов в обоих листьях. Энтропию можно интерпретировать как степень неопределенности, или "загрязненности" (impurity) распределения, и энтропия достигает минимума в том случае, если распределение вероятностей назначает вероятность 1 одному из классов.

Минимизация взвешенной суммы энтропий называется энтропийным критерием разделения. Интересно, что если мы будем минимизировать не кроссэнтропию, а среднюю абсолютную ошибку (MAE) между предсказанным распределением $C_k$ и истинным (эмпирическим) распределением, которое назначает вероятность 1 верному классу, то придем к критерию Джини:

$N_1 \sum\limits_{k=1}^{K} p_k (1 - p_k) + N_2 \sum\limits_{k=1}^{K} p_k^\prime (1 - p_k^\prime) \to \min$

На практике чаще используется энтропийный критерий, потому что он соответствует перекрестной энтропии, которая чаще других применяется в задачах классификации.

**Критерий остановки и обрезка дерева**

Мы рассмотрели шаг роста дерева, но остается еще один вопрос: до какой степени нужно растить дерево? На *рис. 3* показаны шаги роста дерева для классификации по энтропийному критерию. На первых шагах дерево находит явные закономерности в распределении классов, но затем начинает переобучаться на случайном шуме. Если мы соберем ансамбль из деревьев, то проблема переобучения ослабнет. Однако чем больше дерево, тем больше вычислительных ресурсов требуется для его обучения, поэтому размер дерева в ансамбле все же имеет смысл ограничивать.

<img src="assets/trees2.png" width="1000" align="center">

<center><i>Рис. 3. Обучение решающих деревьев в зависимости от максимального количества листьев.</i></center>

На практике чаще ограничивают не суммарное количество листьев, а максимальную глубину дерева, то есть максимальную длину пути от корня к листу. Можно также воспользоваться следующим критерием: прекращать рост дерева, если после добавления нового разделяющего правила функция потерь упала на величину, меньшую некоего порога. Однако такой подход тоже имеет свои проблемы.

На *рис. 4а* показана ситуация, когда добавление первого оптимального разделяющего правила не приведет к значимому снижению ошибки классификации, тогда как следующие два разделяющих правила ведут к радикальному снижению функции потерь. Более того, оптимальное значение порога для первого разделяющего правила сильно зависит от случайных факторов и может быть выбрано не равным нулю. На этом примере мы видим ситуацию "отложенного выигрыша", когда жадный алгоритм построения дерева может сработать не лучшим образом.

<img src="assets/trees3.jpg" width="400" align="center">

<center><i>Рис. 4. Первое решающее правило не даст выигрыша в точности.</i></center>

Обрезкой решающего дерева (pruning) называется процесс удаления из недо отдельных ветвей, которые не приводят к существенному падению функции потерь. Имеел ли смысл сначала строить, а затем удалять ветви? Как мы увидели на *рис. 4*, при построении ветви мы не можем точно сказать, насколько сильно эта ветвь и дочерние к ней ветви помогут снизить функцию потерь. Это станет понятно только тогда, когда мы построим дочерние ветви, затем дочерние к дочерним и так далее. Если даже после этого функцию потерь на данной ветви не удалось сушественно снизить, тогда всю ветвь можно удалить, упростив дерево.

**Работа с категориальными признаками**

До сих пор мы рассматривали работу только с количественными признаками. Если в обучающей выборке $N$ примеров, то в количественном признаке имеет смысл перебирать не более $N-1$ значений порога. Теперь рассмотрим категориальный признак с $K$ категориями. Можно разделить все категории на два подмножества, и отправить эти подмножества в разные ветви. Но количество возможных делений всех категорий на два подмножества растет экспоненциально с ростом количества категорий $K$, что делает невозможным полный перебор при большом $K$.

Есть два достаточно простых пути. Мы можем закодировать категориальный признак one-hot кодированием. Тогда если по этому признаку произойдет деление, то одна категория отправится в одву ветвь, все остальные в другую, то есть мы ищем только деления вида "одина категория против всех". Также мы можем оставить признаки в label-кодировании и рассматривать их как количественный признак, тогда в одну из ветвей отправятся все категории, индексы которых меньше определенного числа. Такие деления получаются менее осмысленными, но из этого напрямую не следует, что эффективность такого подхода будет ниже. Могут быть и другие подходы, например мы можем перебрать какое-то количество случайных делений всех категорий на два подмножества.

На практике установлено, что для категориальных признаков небольшой размерности (т. е. с небольшим количеством категорий) лучше работает one-hot кодирование (см. [Prokhorenkova et al., 2017]($CatBoost: unbiased boosting with categorical features$)). Для признаков большой размерности можно тоже применять one-hot кодирование, хотя если сделать это в явном виде, то получившаяся матрица признаков будет занимать очень много места в памяти. Более эффективным подходом к работе с категориальными признаками большой размерности считается target-кодирование.

При target-кодировании мы заменяем каждую категорию некой статистикой (обычно средним значением) целевой переменной, рассчитанной по объектам данной категории. Например, если категориальной переменной является модель автомобиля, а целевым признаком является цена, то мы рассчитываем среднюю цену по каждой модели, и используем полученные данные вместо модели автомобиля.

Данный подход имеет два недостатка. Во-первых, представим, что в каждой категории только один объект. Тогда после target-кодирования признак будет содержать готовые ответы. Модели, обучаемой на этом датасете, будет достаточно извлекать ответ из этого признака, не используя никакие другие признаки. Очевидно, такой подход приведет к переобучению. Как вариант, можно использовать две обучающие выборки: на первой расссчитывать статистику по целевой переменной, а на второй с помощью этой статистики делать target-кодирование и обучать модель.

Вторая проблема в том, что категориальные признаки могут влиять на целевой признак не независимо друг от друга. Например, пусть мы имеем два категориальных признака в label-кодировани, принимающие значения 0 или 1: если они не равны друг другу, то целевой признак равен 1, в противном случае целевой признак равен 0. Если мы выполним target-кодирование, то эта информация будет полностью утеряна.

Несмотря на эти недостатки, target-кодирование и его различные варианты остается одним из самых эффективных способов работы с категориальными признаками высокой размерности. Авторы библиотеки CatBoost разработали упорядоченное target-кодирование, при котором на обучающих примерах задается некий порядок, и для каждого $i$-го примера статистика по целевой переменной рассчитывается только на основе примеров с индексами меньше $i$ (см. [Prokhorenkova et al., 2017]($CatBoost: unbiased boosting with categorical features$), раздел 3.2). Обзор различных способов target-кодирования также можно найти в [Pargent et al., 2021]($Regularized target encoding outperforms traditional methods in supervised machine learning with high cardinality features$).